# Small Business Loan Approval Analysis

In this notebook, we will analyze the data related to small business loan approvals. Our goal is to understand the factors that influence loan approvals and use machine learning models to predict which businesses should be granted loans.

The analysis will be structured as follows:
1. **Data Preprocessing**: We'll start by loading and cleaning the data, handling missing values, and ensuring it's in the right format for analysis.
2. **Feature Engineering**: We'll create new features that might be relevant for our analysis and predictions.
3. **Exploratory Data Analysis (EDA)**: We'll visualize the data to understand the trends, patterns, and relationships between different features.
4. **Model Building**: We'll train machine learning models to predict loan approvals.
5. **Explainable AI (XAI)**: We'll use SHAP values to understand the importance of different features in our model's predictions.

Let's begin!

In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Setting display options for pandas dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Loading the datasets
file0 = pd.read_csv('file0.csv')
file1 = pd.read_csv('file1.csv')
file2 = pd.read_csv('file2.csv')

# Concatenating the datasets to form a single dataframe
df = pd.concat([file0, file1, file2], ignore_index=True)

# Displaying the first few rows of the dataframe
df.head()

## Data Preprocessing

From the initial glance at the data, we can observe the following:

1. Some columns have data in object format which might be due to the presence of special characters like `$`. We'll need to convert these columns to appropriate numeric formats.
2. The `ChgOffDate` column has 100% missing values in the displayed rows. We'll need to further investigate this column.
3. Some columns like `ApprovalDate`, `DisbursementDate`, and `ChgOffDate` are dates and might need to be converted to datetime format.

Let's start by handling these issues.

In [ ]:
# Converting columns with $ and , to float type
money_columns = ['DisbursementGross', 'BalanceGross', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv']
for col in money_columns:
    df[col] = df[col].str.replace(',', '').str.replace('$', '').astype(float)

# Converting date columns to datetime format
date_columns = ['ApprovalDate', 'DisbursementDate', 'ChgOffDate']
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')

# Checking the percentage of missing values for each column
missing_percentage = df.isnull().mean() * 100
missing_percentage

From the output, we can observe that:

1. The `DisbursementDate` column has about 0.26% missing values.
2. The `BankState` column has about 0.17% missing values.
3. The `Name` column has a very small percentage of missing values (0.0015%).

Given the small percentage of missing values, we can choose to either impute these values or drop the rows with missing values. For simplicity, we'll drop the rows with missing values for now.

Next, we'll proceed with feature engineering.

In [ ]:
# Dropping rows with missing values
df.dropna(subset=['DisbursementDate', 'BankState', 'Name'], inplace=True)

# Resetting the index
df.reset_index(drop=True, inplace=True)

# Checking the shape of the dataframe after dropping missing values
df.shape

## Feature Engineering

Feature engineering involves creating new features or modifying existing features to improve the performance of machine learning models. Given the dataset and the provided examples, we can create the following new features:

1. **Industry**: Based on the first two digits of the `NAICS` feature, we can categorize businesses into their respective industries.
2. **RealEstate**: Based on the `Term` feature, if the term is greater than or equal to 240 months, it indicates that the loan is related to real estate.

Let's create these features.

In [ ]:
# Creating the 'Industry' feature based on the first two digits of 'NAICS'
industry_mapping = {
    '11': 'Agriculture, forestry, fishing, and hunting',
    '21': 'Mining, quarrying, and oil and gas extraction',
    '22': 'Utilities',
    '23': 'Construction',
    '31': 'Manufacturing', '32': 'Manufacturing', '33': 'Manufacturing',
    '42': 'Wholesale trade',
    '44': 'Retail trade', '45': 'Retail trade',
    '48': 'Transportation and warehousing', '49': 'Transportation and warehousing',
    '51': 'Information',
    '52': 'Finance and insurance',
    '53': 'Real estate and rental and leasing',
    '54': 'Professional, scientific, and technical services',
    '55': 'Management of companies and enterprises',
    '56': 'Administrative and support and waste management and remediation services',
    '61': 'Educational services',
    '62': 'Health care and social assistance',
    '71': 'Arts, entertainment, and recreation',
    '72': 'Accommodation and food services',
    '81': 'Other services except public administration',
    '92': 'Public administration'
}
df['Industry'] = df['NAICS'].astype(str).str[:2].map(industry_mapping)

# Creating the 'RealEstate' feature based on the 'Term' feature
df['RealEstate'] = np.where(df['Term'] >= 240, 1, 0)

# Displaying the first few rows of the dataframe after feature engineering
df[['NAICS', 'Industry', 'Term', 'RealEstate']].head()

The new features `Industry` and `RealEstate` have been successfully created. We can observe that some rows have `NAICS` as 0, which results in NaN values for the `Industry` feature. We'll handle these NaN values by categorizing them as 'Unknown'.

Next, we'll proceed with exploratory data analysis (EDA) to visualize and understand the data better.

In [ ]:
# Handling NaN values in 'Industry' column
df['Industry'].fillna('Unknown', inplace=True)

# Checking the unique values in 'Industry' column
df['Industry'].value_counts()

## Exploratory Data Analysis (EDA)

EDA helps us understand the data by visualizing and summarizing its main characteristics. This step is crucial before diving into machine learning or statistical modeling as it allows us to understand the underlying structure of the data, identify outliers, and detect anomalies.

Let's start by visualizing the distribution of loans across different industries.

In [ ]:
# Visualizing the distribution of loans across different industries
plt.figure(figsize=(15, 10))
industry_counts = df['Industry'].value_counts()
sns.barplot(y=industry_counts.index, x=industry_counts.values, palette='viridis')
plt.xlabel('Number of Loans')
plt.ylabel('Industry')
plt.title('Distribution of Loans Across Different Industries')
plt.show()

In [ ]:
# Continuing the creation of the 'RealEstate' feature based on the 'Term' feature
df['RealEstate'] = np.where(df['Term'] >= 240, 1, 0)

# Displaying the first few rows of the dataframe to check the new features
df[['NAICS', 'Industry', 'Term', 'RealEstate']].head()

The new features, `Industry` and `RealEstate`, have been successfully created. We can observe that some rows have 'Unknown' as their industry. This is because the `NAICS` code for these rows is either missing or doesn't match the provided mapping. We'll handle these 'Unknown' values in the next steps.

Next, let's proceed with Exploratory Data Analysis (EDA) to visualize and understand the data better.

## Exploratory Data Analysis (EDA)

EDA helps in understanding the data by visualizing and summarizing its main characteristics. We'll start by visualizing the distribution of loan approvals based on different features to identify patterns and trends.

In [ ]:
# Plotting the distribution of loan approvals based on the 'Industry' feature
plt.figure(figsize=(15, 8))
sns.countplot(y='Industry', hue='MIS_Status', data=df, order=df['Industry'].value_counts().index)
plt.title('Distribution of Loan Approvals based on Industry')
plt.xlabel('Count')
plt.ylabel('Industry')
plt.legend(title='Loan Status')
plt.tight_layout()
plt.show()

From the above visualization, we can observe the following:

1. **Retail Trade** and **Professional, scientific, and technical services** are the top industries seeking loans.
2. Most of the loans across all industries are paid in full (`PIF`). However, there are still a significant number of loans that are charged off (`CHGOFF`), especially in the **Retail Trade** and **Construction** industries.
3. The **Unknown** category has a high number of loans, indicating that there are many businesses with NAICS codes that don't match the provided mapping. We might need to further investigate or categorize these businesses in future analyses.

Next, let's visualize the distribution of loan approvals based on the `RealEstate` feature.

In [ ]:
# Plotting the distribution of loan approvals based on the 'RealEstate' feature
plt.figure(figsize=(10, 6))
sns.countplot(x='RealEstate', hue='MIS_Status', data=df)
plt.title('Distribution of Loan Approvals based on RealEstate')
plt.xlabel('RealEstate (0: Non-RealEstate, 1: RealEstate)')
plt.ylabel('Count')
plt.legend(title='Loan Status')
plt.tight_layout()
plt.show()

From the visualization, we can observe the following:

1. A majority of the loans are not related to real estate (`RealEstate = 0`), but there's still a significant number of real estate-related loans.
2. For both real estate and non-real estate loans, most of the loans are paid in full (`PIF`). However, the proportion of charged off loans (`CHGOFF`) seems slightly higher for non-real estate loans.

This indicates that real estate loans might be slightly less risky, possibly because the underlying asset (real estate) can be used as collateral.

Next, we'll proceed with building machine learning models to predict loan approvals.

## Model Building

We'll start by preparing the data for modeling. This involves:

1. Encoding categorical variables.
2. Splitting the data into training and testing sets.
3. Scaling the features.

After preparing the data, we'll train a few machine learning models and evaluate their performance. Given the CPU limit, we'll select models that are computationally efficient.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Selecting relevant features for modeling
features = ['City', 'State', 'Zip', 'Bank', 'BankState', 'NAICS', 'ApprovalFY', 'Term', 'NoEmp', 'NewExist',
            'CreateJob', 'RetainedJob', 'UrbanRural', 'RevLineCr', 'LowDoc', 'DisbursementGross',
            'BalanceGross', 'GrAppv', 'SBA_Appv', 'Industry', 'RealEstate']
target = 'MIS_Status'

# Encoding categorical variables
label_encoders = {}
for col in df[features].select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=42)

# Scaling the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train.shape, X_test.shape

The data has been successfully preprocessed and split into training and testing sets. We have 716,216 samples in the training set and 179,055 samples in the testing set, with each sample having 21 features.

Next, we'll train a few machine learning models. Given the CPU limit, we'll start with logistic regression, which is computationally efficient. We'll then evaluate its performance on the testing set.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Initializing and training the logistic regression model
logreg = LogisticRegression(max_iter=1000, random_state=42)
logreg.fit(X_train, y_train)

# Predicting on the testing set
y_pred = logreg.predict(X_test)

# Evaluating the model's performance
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

accuracy, classification_rep

It seems there's an error due to the presence of NaN values in the input data. We'll need to investigate and handle these NaN values before proceeding with the model training.

In [ ]:
# Checking for NaN values in the training set
nan_columns = X_train.columns[np.any(np.isnan(X_train), axis=0)]
nan_columns

In [ ]:
# Continuing the training of the logistic regression model
logreg.fit(X_train, y_train)

# Predicting on the test set
y_pred = logreg.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

accuracy, classification_rep

In [ ]:
# Checking for NaN values in the training and testing sets
nan_train = pd.DataFrame(X_train).isna().sum().sum()
nan_test = pd.DataFrame(X_test).isna().sum().sum()

nan_train, nan_test

In [ ]:
from sklearn.impute import SimpleImputer

# Imputing NaN values with the mean of the respective columns
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Retraining the logistic regression model
logreg.fit(X_train, y_train)

# Predicting on the test set
y_pred = logreg.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

accuracy, classification_rep

In [ ]:
# Checking for NaN values again in the imputed training and testing sets
nan_train_imputed = pd.DataFrame(X_train).isna().sum().sum()
nan_test_imputed = pd.DataFrame(X_test).isna().sum().sum()

nan_train_imputed, nan_test_imputed

In [ ]:
# Checking for NaN values in the target variable
nan_y_train = y_train.isna().sum()
nan_y_test = y_test.isna().sum()

nan_y_train, nan_y_test

In [ ]:
# Removing rows with NaN values in the target variable
X_train = pd.DataFrame(X_train)[~y_train.isna()].values
y_train = y_train.dropna().values

X_test = pd.DataFrame(X_test)[~y_test.isna()].values
y_test = y_test.dropna().values

# Retraining the logistic regression model
logreg.fit(X_train, y_train)

# Predicting on the test set
y_pred = logreg.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

accuracy, classification_rep

In [ ]:
# Resetting indices of y_train and y_test
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# Removing rows with NaN values in the target variable
X_train = pd.DataFrame(X_train)[~y_train.isna()].values
y_train = y_train.dropna().values

X_test = pd.DataFrame(X_test)[~y_test.isna()].values
y_test = y_test.dropna().values

# Retraining the logistic regression model
logreg.fit(X_train, y_train)

# Predicting on the test set
y_pred = logreg.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

accuracy, classification_rep

## Feature Importance using SHAP Values

SHAP (SHapley Additive exPlanations) values provide a measure of the impact of each feature on the model's prediction for a given instance. By aggregating these values across all instances, we can get a sense of the overall importance of each feature in the model.

Let's compute and visualize the SHAP values for our logistic regression model to understand which features are most influential in determining loan approvals.

In [ ]:
!pip install -q shap

In [ ]:
import shap

# Initializing the explainer
explainer = shap.LinearExplainer(logreg, X_train, feature_perturbation="interventional")

# Calculating SHAP values
shap_values = explainer.shap_values(X_test)

# Visualizing the SHAP values
shap.summary_plot(shap_values, X_test, feature_names=features, plot_type="bar")